# **Module `pepper.rle`**

✔ Revue des **typehints** et des **docstrings**.

# **`row_rle`**`(row: pd.Series) -> np.ndarray`

DEPRECATED

# **`series_rle_reduction`**`(s: pd.Series) -> np.ndarray`

# **`data_rle_reduction`**`(data: pd.DataFrame) -> pd.Series`

# **`jumps_rle`**`(s: pd.Series) -> tuple`

In [ ]:
from pepper.rle import jumps_rle
import pandas as pd

s = pd.Series([1, 2, 3, 4, 5, 6])
print(jumps_rle(s))
# ((1, 5),)

s = pd.Series([1, 1, 1, 2, 2, 3])
print(jumps_rle(s))
# ((0, 2), (1, 1), (0, 1), (1, 1))

((1, 5),)
((0, 2), (1, 1), (0, 1), (1, 1))


# Note à propos des arrondis et des groupby qui forcent le passage par le 64 bits

**TODO** : régler les problèmes d'arrondi dans le RLE, certainement liés à mon choix de réduction de la taille d'encodage : correction dans RLE réduction, avec l'ajout d'un paramètre de précision, qui est d'autant plus important qu'il permet de mieux synthétiser à mesure que la précision diminue.

Après analyse : c'est le `groupby` et la gestion des types non `float64` par pandas qui est en cause: les valeurs de mes groupes en `float32` ont changé à la sortie et sont des valeurs dégénérées (`0.22` est devenu `0.2199999988079071` alors que le groupement devrait se contenter de collecter les valeurs sans les modifier). La raison est la suivante : le groupby force le passage par np.float64 : en effet,`np.float4(0.22) = 0.2199999988079071` 

Les stratégies pour contourner :
1. ajouter un paramètre optionnel `decimals` à mon `series_rle_reduction` pour qu'il retraite via un `np.round` mais valeurs dégénérées par le `groupby` : c'est également une extension fonctionnelle, puisque permet de regrouper des termes proches.
2. moins intrusif et coûteux, simplement corriger le transtypage non sollicité imposé par le `groupby`, avec le passage du `dtype` d'origine à `series_rle_reduction`.
3. Avec un temps imparti réduit, rester sur du `float64` dans les tables de niveaux 2 et 3, et ne downcaster que dans le merge final.

## Work in progress : Expansion RLE

Du codage RLE au dataframe.

In [1]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status_by_client_and_month()
display(status)

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH  STATUS
SK_ID_CURR MONTHS_BALANCE                     
100001     0                              0.22
           1                              0.00
           2                              0.00
           3                              0.00
           4                              0.00
...                                        ...
456255     72                             0.00
           73                             0.00
           74                             0.00
           75                             0.00
           76                             0.00

[7365660 rows x 1 columns]

In [2]:
import numpy as np
x = status.loc[100001]["STATUS"].iloc[0]
display(x)
print(type(x))
print(np.float64(x))
print(np.float32(np.float64(x)))

0.22

<class 'numpy.float32'>
0.2199999988079071
0.22


In [3]:
s = status.loc[100001]["STATUS"]

display(s.values)

array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ], dtype=float32)

In [4]:
from pepper.rle import series_rle_reduction
r = series_rle_reduction(s)
print(r)
print(type(r))

((0.2199999988079071, 1), (0.0, 51))
<class 'tuple'>


In [5]:
import numpy as np
r = ((0, 1), (1, 2), (2, 3))
display(np.repeat(*(np.array(r).T)))

array([0, 1, 1, 2, 2, 2])

In [6]:
r = series_rle_reduction(s)
a = np.array(r)
display(a)
values, counts = np.split(np.array(r), 2, axis=1)
counts = counts.astype(int)
print(values[:, 0])
print(counts[:, 0])
display(np.repeat(values[:, 0], counts[:, 0]))

array([[ 0.22,  1.  ],
       [ 0.  , 51.  ]])

[0.22 0.  ]
[ 1 51]


array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [4]:
from pepper.rle import rle_expand_expr

display(rle_expand_expr(((0, 1), (1, 2), (2, 3))))
display(rle_expand_expr([[1, 1], [0, 2], [.5, 1]]))

array([0, 1, 1, 2, 2, 2])

array([1. , 0. , 0. , 0.5])

In [1]:
from home_credit.tables import BureauBalance

activity_variation = BureauBalance.rle_loan_activity_by_client_variation()
display(activity_variation)

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH MONTHS_BALANCE            \
                                                    min max count   
SK_ID_CURR                                                          
100001                                                0  51    52   
100002                                                0  47    48   
100005                                                0  12    13   
100010                                                2  90    72   
100013                                                0  68    69   
...                                                 ...  ..   ...   
456247                                                0  81    82   
456250                                                0  32    33   
456253                                                0  30    31   
456254                                                0  36    37   
456255                                                0  76    77   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                       \
                                                                    jumps_rle   
SK_ID_CURR                                                                      
100001                                                              [[1, 52]]   
100002                                                              [[1, 48]]   
100005                                                              [[1, 13]]   
100010                                    [[3, 1], [1, 35], [18, 1], [1, 35]]   
100013                                                              [[1, 69]]   
...                                                                       ...   
456247                                                              [[1, 82]]   
456250                                                              [[1, 33]]   
456253                                                              [[1, 31]]   
456254                                                              [[1, 37]]   
456255                                                              [[1, 77]]   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                             ACTIVE  
                                                                       series_rle_reduction  
SK_ID_CURR                                                                                   
100001                                    [[3, 2], [2, 9], [1, 10], [2, 8], [1, 1], [0, ...  
100002                                    [[1, 4], [0, 9], [1, 3], [0, 1], [1, 1], [2, 3...  
100005                                                                    [[2, 3], [1, 10]]  
100010                                                 [[0, 26], [1, 10], [0, 26], [1, 10]]  
100013                                    [[1, 18], [2, 22], [1, 1], [2, 3], [3, 11], [2...  
...                                                                                     ...  
456247                                    [[2, 1], [3, 2], [4, 7], [3, 1], [2, 1], [1, 1...  
456250                                                    [[2, 25], [3, 1], [2, 2], [1, 5]]  
456253                                                            [[1, 19], [4, 5], [3, 7]]  
456254                                                                    [[0, 29], [1, 8]]  
456255                                    [[0, 3], [2, 4], [3, 3], [4, 2], [3, 1], [2, 2...  

[134542 rows x 5 columns]

In [2]:
# display(activity_variation)
rle_series = activity_variation[("ACTIVE", "series_rle_reduction")]
display(rle_series)

SK_ID_CURR
100001    [[3, 2], [2, 9], [1, 10], [2, 8], [1, 1], [0, ...
100002    [[1, 4], [0, 9], [1, 3], [0, 1], [1, 1], [2, 3...
100005                                    [[2, 3], [1, 10]]
100010                 [[0, 26], [1, 10], [0, 26], [1, 10]]
100013    [[1, 18], [2, 22], [1, 1], [2, 3], [3, 11], [2...
                                ...                        
456247    [[2, 1], [3, 2], [4, 7], [3, 1], [2, 1], [1, 1...
456250                    [[2, 25], [3, 1], [2, 2], [1, 5]]
456253                            [[1, 19], [4, 5], [3, 7]]
456254                                    [[0, 29], [1, 8]]
456255    [[0, 3], [2, 4], [3, 3], [4, 2], [3, 1], [2, 2...
Name: (ACTIVE, series_rle_reduction), Length: 134542, dtype: object

In [5]:
from pepper.rle import rle_expand_expr

# TODO Fix it. régression certainement due au passage de tuple à liste forcé par l'AR parquet

display(rle_series.apply(rle_expand_expr))

IndexError: tuple index out of range

In [ ]:
np.repeat([1, np.nan, 2], [1, 2, 3])

array([ 1., nan, nan,  2.,  2.,  2.])

Il est facile d'étendre une series de rle_expr comme cela : rle_series.apply(rle_expand_expr)

Mais j'ai fait quelque chose d'assez fin : mes expressions rle résument les cellules non NA d'un tableau à 96 dimensions.

Avec une seconde fonction rle_jumps, je code le support, c'est à dire les indices des positions successives non NA, en partant (prepend) de -1 : par exemple (1, 3)(3, 1)(1, 3) donnera les indices suivants : 0, 1, 2, 5, 6, 7, 8. Mon but est de faire une fonction qui prend en argument deux series, celle qui code les supports et celle qui code les valeurs, et de produire en sortie un dataframe.

La première étape me semble d'être d'identifier la dimension du dataframe de sortie, qui est finalement donné par le support le plus long. Donc la première première étape est d'extraire cette information d'un codage rle de support, ce qui n'est pas très compliqué.

Ensuite, il faut trouver la succession d'opérations la plus efficace pour réaliser l'expansion.

Pour tirer parti de la fonction précédente, basée sur repeat, donc efficace, je pense que la meilleure stratégie est de modifier les rle_expr en fonction des rle_jumps, pour y insérer des 'trous', c'est à dire des répétitions de np.nan là où le support l'indique. Enfin, on utilie notre fonction précédente avec un apply sur les rle_expr ainsi transformée. 

In [ ]:
jumps_rle_series = activity_variation[("MONTHS_BALANCE", "jumps_rle")]
display(jumps_rle_series)

SK_ID_CURR
100001                             ((1, 52),)
100002                             ((1, 48),)
100005                             ((1, 13),)
100010    ((3, 1), (1, 35), (18, 1), (1, 35))
100013                             ((1, 69),)
                         ...                 
456247                             ((1, 82),)
456250                             ((1, 33),)
456253                             ((1, 31),)
456254                             ((1, 37),)
456255                             ((1, 77),)
Name: (MONTHS_BALANCE, jumps_rle), Length: 134542, dtype: object

In [ ]:
from pepper.rle import support_max_indice

print(support_max_indice(((1, 52),)))
print(support_max_indice(((3, 1), (1, 35), (18, 1), (1, 35))))
max_indices = jumps_rle_series.apply(support_max_indice)
output_dim = max_indices.max() + 1
print(f"max_max_indice + 1 = dim: {output_dim}")
display(max_indices)

51
90
max_max_indice + 1 = dim: 97


SK_ID_CURR
100001    51
100002    47
100005    12
100010    90
100013    68
          ..
456247    81
456250    32
456253    30
456254    36
456255    76
Name: (MONTHS_BALANCE, jumps_rle), Length: 134542, dtype: int64

Idée :

Avec un support $(1, N)$, il n'y a rien à faire, séquence continue, pas de trous. L'opération de transformation de la rle_values est l'identité (c'est le cas le plus fréquent en la circonstance de nos données Home Credit).

Avec un support fragmenté, de la forme $(1, i)(k, j)(1, l)$ : il y a un trou de taille $k-1$ après la séquence des $i$ premières valeurs, suivi des $l+1$ valeurs suivantes.

Avec $k_j > 1$,

Avec un support $(1, i_1)(k_1, j_1)(1, i_2)(k_2, j_2)\ldots(1, i_n)(k_n, j_n)(1, i_{n+1})$, il y a $n$ _trous_ de tailles $k_1-1, \dots, k_n-1$, précédés de $i_1, i_2+1, i_3+1, \ldots, i_n+1$ valeurs.

Attention au cas où cela démarre par un trou !

Cela revient donc à insérer $n$ `(np.nan, k_j-1)` dans l'expression RLE des valeurs.

Là il faut réfléchir un peu pour le faire en Numpy, mais en gros, les distances d'insertion sont données par les $i_1, i_2+1, i_3+1, \ldots, i_n+1$. Par exemple, pour la première insertion, on identifie dans la séquence de valeurs, l'élément répété, tel que $i_1$ soit supérieur strictement à la somme des répétitions des éléments précédents, mais inférieure ou égale à cette somme augmentée du nombre de répétitions de cet élément.

Là, il faut éventuellement fragmenter l'élément en deux parties, celle qui précède le trou et celle qui le suit.

On procède à l'insertion.

La suite et une répétition de ce procédé.

In [ ]:
from pepper.rle import rle_expr_to_numpy
import numpy as np

support = ((1, 3), (3, 1), (1, 3))
values = ((1, 1), (2, 3), (3, 1))
def n_holes(rle_support: np.array):
    return np.sum(rle_support[:, 0] > 1)

rle_support = rle_expr_to_numpy(support)
rle_values = rle_expr_to_numpy(values)
print(f"support: {support}")
print(f"values: {values}")

print(n_holes(rle_support))

hole_sizes = rle_support[rle_support[:, 0] > 1, 0] - 1
print(f"hole sizes (k-1): {hole_sizes}")

start_with_hole = rle_support[0, 0] > 1
print(f"start with hole: {start_with_hole}")

seq_sizes = rle_support[rle_support[:, 0] == 1, 0] + 1
if not start_with_hole:
    seq_sizes[0] -= 1
print(f"seq sizes (i+1): {seq_sizes}")

vc_cumsum = np.cumsum(rle_values[:, 1])
print(f"value counts cumsum: {vc_cumsum}")


support: ((1, 3), (3, 1), (1, 3))
values: ((1, 1), (2, 3), (3, 1))
1
hole sizes (k-1): [2]
start with hole: False
seq sizes (i+1): [1 2]
value counts cumsum: [1 4 5]


In [ ]:
from typing import Union
from pepper.rle import rle_support_size


# TODO : mettre au point plus tard, il y a des urgences, et cela est du bonus.

def rle_insert_holes(
    rle_support_expr: Union[list, tuple, np.ndarray],
    rle_values_expr: Union[list, tuple, np.ndarray]
) -> tuple:
    """
    Modify a RLE sequence of values expression by inserting "holes" based on support.

    Parameters
    ----------
    rle_support_expr : List[tuple]
        RLE sequence for support.
    rle_values_expr : List[tuple]
        RLE sequence for values.

    Returns
    -------
    tuple
        RLE sequence of values expression with holes.
    """
    # Convert inputs to NumPy arrays if they are not already
    rle_support = rle_expr_to_numpy(rle_support_expr)
    rle_values = rle_expr_to_numpy(rle_values_expr)

    # Ensure that both support and values have the same length
    # if rle_support.shape[0] != rle_values.shape[0]:
    if rle_support_size(rle_support) != rle_support_size(rle_values):
        raise ValueError("Number of repeats mismatch between the support and values RLE sequences.")

    # Count holes
    h = n_holes(rle_support)
    
    # No holes, keep values as is
    if not h:
        return rle_values_expr
    
    
    
    ## La suite est a early bad..
    
    modified_values.append(values)

    # Create an empty list to store the modified RLE values
    modified_values = []

    for i in range(rle_support.shape[0]):
        support = rle_support[i]
        values = rle_values[i]

        if support[0] == 1:
            # No holes, keep values as is
            modified_values.append(values)
        else:
            # Calculate the number of holes and their positions
            num_holes = support[0] - 1
            hole_positions = np.cumsum([values[1] + num_holes] + list(support[1:]))

            # Split values into segments before and after holes
            segments = np.split(values, hole_positions[:-1])
            modified_values.extend(segments[:-1])

            # Insert NaN holes between segments
            modified_values.extend([(np.nan, num_holes)] * num_holes)

    # Convert the list of modified values to a NumPy array
    modified_values_array = np.concatenate(modified_values)

    return rle_support, modified_values_array

# Example usage:
rle_support_expr = ((1, 2), (3, 1), (1, 3))
rle_values_expr = ((0, 1), (1, 2), (2, 3))
result_support, result_values = rle_insert_holes(rle_support_expr, rle_values_expr)
print(result_support)
print(result_values)


[[1 2]
 [3 1]
 [1 3]]
[ 0.  1.  1.  2. nan  2. nan  2.  2.  3.]
